<a href="https://colab.research.google.com/github/chaeyh4/pm2.5/blob/main/final_aispark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install numpy pandas tensorflow keras sklearn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
import os
import glob
from pandas._libs.tslibs import tz_convert_from_utc_single

os.chdir('/content/awsdata')
all_files = glob.glob('*.csv')
dfs = []
for file in all_files:
    df = pd.read_csv(file)
    dfs.append(df)

aws_data = pd.concat(dfs, ignore_index=True)
aws_data.to_csv('/content/new/combined_aws_data.csv', index=False)

os.chdir('/content/pmdata')  
my_files = glob.glob('*.csv')
dfss = []
for file in my_files:
    dfp = pd.read_csv(file)
    dfss.append(dfp)

pm_data = pd.concat(dfss, ignore_index=True)
pm_data.to_csv('/content/new/combined_pm_data.csv', index=False)


new_column = {'측정소' : '지점'}
pm_data = pm_data.rename(columns= new_column)

os.chdir('/content/aws station')
aws_stations = pd.read_csv('awsmap.csv')

os.chdir('/content/pm station')
pm_stations = pd.read_csv('pmmap.csv')

drop_pm_stat = ['Description']
pm_stations = pm_stations.drop(columns=drop_pm_stat, axis = 1)
new_column = {'Location' : '지점',
              'Latitude' : '위도',
              'Longitude' : '경도'}
pm_stations = pm_stations.rename(columns= new_column)

drop_aws_stat = ['Description']
aws_stations = aws_stations.drop(columns=drop_aws_stat, axis = 1)
new_column = {'Location' : '지점',
              'Latitude' : '위도',
              'Longitude' : '경도'}
aws_stations = aws_stations.rename(columns= new_column)

def convert_year(year):
    return str(2000 + int(year))

pm_data['관측시점'] = pm_data['연도'].apply(convert_year) + '-' + pm_data['일시']
aws_data['관측시점'] = aws_data['연도'].apply(convert_year) + '-' + aws_data['일시']

pm_data = pm_data.merge(pm_stations, on='지점', how='left')
aws_data = aws_data.merge(aws_stations, on='지점', how='left')

pm_data.head(5)

# Remove rows with any null values in pm_data
clean_pm_data = pm_data.dropna()
clean_pm_data.to_csv('/content/clean_combined_pm_data.csv', index=False)

# Remove rows with any null values in aws_data
clean_aws_data = aws_data.dropna()
clean_aws_data.to_csv('/content/clean_combined_aws_data.csv', index=False)

import pandas as pd
from sklearn.neighbors import NearestNeighbors

# Extract unique locations and coordinates for both datasets
pm_locations = clean_pm_data[['지점', '위도', '경도']].drop_duplicates()
aws_locations = clean_aws_data[['지점', '위도', '경도']].drop_duplicates()

# Find the nearest AWS location for each PM location using NearestNeighbors
nearest_neighbors = NearestNeighbors(n_neighbors=1)
nearest_neighbors.fit(aws_locations[['위도', '경도']].values)

distances, indices = nearest_neighbors.kneighbors(pm_locations[['위도', '경도']].values)

# Map the nearest AWS location to the corresponding PM location
pm_locations['nearest_aws_지점'] = aws_locations.iloc[indices.flatten()]['지점'].values

# Merge the PM data with the nearest AWS data
merged_data = clean_pm_data.merge(pm_locations[['지점', 'nearest_aws_지점']], on='지점')
merged_data = merged_data.merge(clean_aws_data, left_on=['nearest_aws_지점', '연도', '일시'], right_on=['지점', '연도', '일시'], suffixes=('', '_aws'))

# Drop unnecessary columns from the merged DataFrame
merged_data.drop(['지점_aws', '위도_aws', '경도_aws'], axis=1, inplace=True)

print(merged_data.head())

# Create a new DataFrame to store the PM location, its coordinates,
# the nearest AWS location, and the coordinates of the nearest AWS location
location_mapping = pm_locations[['지점', '위도', '경도']].copy()
location_mapping['nearest_aws_지점'] = pm_locations['nearest_aws_지점']

# Merge the coordinates of the nearest AWS locations
location_mapping = location_mapping.merge(
    aws_locations[['지점', '위도', '경도']],
    left_on='nearest_aws_지점',
    right_on='지점',
    suffixes=('_pm', '_aws')
)

# Rename the columns for clarity
location_mapping.columns = [
    '지점_pm', '위도_pm', '경도_pm',
    'nearest_aws_지점', '지점_aws', '위도_aws', '경도_aws'
]

print(location_mapping)

df = merged_data.drop(columns=['관측시점_aws', '위도', '경도', '연도', '일시'])

def reorder_columns(df, col_name):
    cols = df.columns.tolist()
    cols.remove(col_name)
    cols.append(col_name)
    return df[cols]

df = reorder_columns(df, 'nearest_aws_지점')
df = merged_data.drop(columns=['nearest_aws_지점'])

# Sort the DataFrame by 'year-date-time' and 'location' columns
sorted_data = df.sort_values(by=['지점','관측시점'])
sorted_data = sorted_data.drop(columns=['관측시점_aws', '위도', '경도', '연도', '일시'])
# Display the sorted DataFrame
sorted_data.head(100)

import pandas as pd
import os
import glob

def fill_missing_values(df):
    first_valid_value = df['지점'].first_valid_index()
    if first_valid_value is not None:
        df['지점'] = df['지점'].fillna(df.loc[first_valid_value, '지점'])
    return df

os.chdir('/content/aws_test')
all_files = glob.glob('*.csv')
dft = []
for file in all_files:
    dfk = pd.read_csv(file)
    dfk_filled = fill_missing_values(dfk)
    dft.append(dfk_filled)

aws_test = pd.concat(dft, ignore_index=True)

os.chdir('/content/pm_test')
my_files = glob.glob('*.csv')
dftt = []
for file in my_files:
    dfl = pd.read_csv(file)
    dftt.append(dfl)

pm_test = pd.concat(dftt, ignore_index=True)

new_column = {'측정소' : '지점'}
pm_test = pm_test.rename(columns= new_column)

def convert_year(year):
    return str(2000 + int(year))

pm_test['관측시점'] = pm_test['연도'].apply(convert_year) + '-' + pm_test['일시']
aws_test['관측시점'] = aws_test['연도'].apply(convert_year) + '-' + aws_test['일시']

pm_test = pm_test.merge(pm_stations, on='지점', how='left')
aws_test = aws_test.merge(aws_stations, on='지점', how='left')

aws_test.head(100)

import pandas as pd
from sklearn.neighbors import NearestNeighbors


pmtest_locations = pm_test[['지점', '위도', '경도']].drop_duplicates()
awstest_locations = aws_test[['지점', '위도', '경도']].drop_duplicates()

# Find the nearest AWS location for each PM location using NearestNeighbors
nearest_neighbors = NearestNeighbors(n_neighbors=1)
nearest_neighbors.fit(awstest_locations[['위도', '경도']].values)

distances, indices = nearest_neighbors.kneighbors(pmtest_locations[['위도', '경도']].values)

# Map the nearest AWS location to the corresponding PM location
pmtest_locations['nearest_aws_지점'] = awstest_locations.iloc[indices.flatten()]['지점'].values

# Merge the PM data with the nearest AWS data
merge_data = pm_test.merge(pmtest_locations[['지점', 'nearest_aws_지점']], on='지점')
merge_data = merge_data.merge(aws_test, left_on=['nearest_aws_지점', '연도', '일시'], right_on=['지점', '연도', '일시'], suffixes=('', '_aws'))

# Drop unnecessary columns from the merged DataFrame
merge_data.drop(['지점_aws', '위도_aws', '경도_aws'], axis=1, inplace=True)

print(merge_data.head())

dff = merge_data.drop(columns=['관측시점_aws', '위도', '경도', '연도', '일시'])
dff = merge_data.drop(columns=['nearest_aws_지점'])

sort_data = dff.sort_values(by=['지점','관측시점'])
sort_data = sort_data.drop(columns=['관측시점_aws', '위도', '경도', '연도', '일시'])
sort_data.head(2000)
print(sort_data)

import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error
from tensorflow.keras.regularizers import L1L2
import tensorflow as tf

np.random.seed(70)
tf.random.set_seed(70)

def has_missing_values(array):
    return np.isnan(array).any()

def prepare_data(data, lookback, future):
    X, Y = [], []
    for i in range(len(data) - lookback - future):
        X.append(data[i : (i + lookback), :])
        Y.append(data[(i + lookback) : (i + lookback + future), 0])  
    return np.array(X), np.array(Y)

def prepare_data_with_missing(data, lookback, future):
    X, Y, missing_indices = [], [], []
    for i in range(len(data) - lookback - future):
        input_sequence = data[i : (i + lookback), :]
        output_sequence = data[(i + lookback) : (i + lookback + future), 0]

        if has_missing_values(input_sequence):
            continue

        X.append(input_sequence)

        if has_missing_values(output_sequence):
            missing_indices.append(i)
        else:
            Y.append(output_sequence)

    return np.array(X), np.array(Y), missing_indices

lookback = 2 * 24  # 2 days of hourly data
future = 3 * 24    # 3 days of hourly data
n_features = 6     # Number of features in the data


locations = ['공주', '노은동', '논산', '대천2동', '독곶리', '동문동', '모종동', '문창동', '성성동', '신방동', '신흥동', '아름동', '예산군', '읍내동', '이원면', '정림동', '홍성읍']  # List all 17 locations
#locations = ['공주']

all_test_data_real = pd.DataFrame()

for location in locations:
    train_data_loc = sorted_data[sorted_data['지점'] == location]
    test_data_loc = sort_data[sort_data['지점'] == location]
    
    # Drop the '지점' and '관측시점' columns
    train_data_loc = train_data_loc.drop(['지점', '관측시점'], axis=1).values
    test_data_loc = test_data_loc.drop(['지점', '관측시점'], axis=1).values
    
    # Scale the data
    scaler = MinMaxScaler()
    train_data_loc = scaler.fit_transform(train_data_loc)
    test_data_loc = scaler.transform(test_data_loc)

    # Prepare the data for LSTM
    X_train, Y_train = prepare_data(train_data_loc, lookback, future)
    X_test, Y_test, missing_indices = prepare_data_with_missing(test_data_loc, lookback, future)
    
    # Create the LSTM model
    model = Sequential()
    model.add(LSTM(256, activation='tanh', input_shape=(lookback, n_features), return_sequences=True))
    model.add(Dropout(0.2))  # Add dropout with a dropout rate of 0.2
    model.add(LSTM(256, activation='tanh'))
    model.add(Dropout(0.2))  # Add dropout with a dropout rate of 0.2
    model.add(Dense(future))
    model.compile(optimizer='adam', loss='mae')

    # Add EarlyStopping callback
    early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

    # Train the model with validation split
    history = model.fit(X_train, Y_train, epochs=30, verbose=0, validation_split=0.2, callbacks=[early_stopping])
    # Get the train and validation losses
    train_loss = history.history['loss']
    val_loss = history.history['val_loss']

    # Make predictions
    Y_train_pred = model.predict(X_train)
    Y_val_pred = model.predict(X_train[-int(0.2 * len(X_train)):])

    # Calculate the mean absolute error
    train_mae = mean_absolute_error(Y_train, Y_train_pred)
    val_mae = mean_absolute_error(Y_train[-int(0.2 * len(Y_train)):], Y_val_pred)

    print(f"Location: {location}, Train MAE: {train_mae}, Validation MAE: {val_mae}")

    Y_test_missing_pred = model.predict(X_test)

    for i, missing_index in enumerate(missing_indices):
        start = missing_index + lookback
        end = start + future
        test_data_loc[start:end, 0] = Y_test_missing_pred[i]

    # Inverse transform the test data to get real values
    test_data_loc_real = scaler.inverse_transform(test_data_loc)

    # Create a new DataFrame with real values
    test_data_loc_real_df = pd.DataFrame(test_data_loc_real, columns=['PM2.5', 'tem', 'wind degree', 'wind velocity', 'precipitation', 'humidity'])

    # Get the '지점' and '관측시점' columns from the original test data
    test_data_loc_observation = sort_data.loc[sort_data['지점'] == location, ['지점', '관측시점']].reset_index(drop=True)

    # Concatenate '지점' and '관측시점' columns with the test_data_loc_real_df DataFrame
    test_data_loc_real_df = pd.concat([test_data_loc_observation, test_data_loc_real_df], axis=1)

    # Append the test_data_loc_real_df to the all_test_data_real DataFrame
    all_test_data_real = all_test_data_real.append(test_data_loc_real_df, ignore_index=True)    

# Sort the DataFrame by '지점' and '관측시점'
all_test_data_real = all_test_data_real.sort_values(by=['지점', '관측시점']).reset_index(drop=True)

# Print the resulting DataFrame
all_test_data_real.to_csv('/content/new/result_256_drop3.csv', index=False)

all_test_data_real.head(100)

result = all_test_data_real
new_column = {'일시' : '관측시점'}
result = result.rename(columns= new_column)
result
result.to_csv('/content/new/again.csv', index=False)
result.head(50)

os.chdir('/content/sample')
all_files = glob.glob('*.csv')
dfs = []
for file in all_files:
    df = pd.read_csv(file)
    dfs.append(df)

answer = pd.concat(dfs, ignore_index=True)
new_column = {'측정소' : '지점'}
answer = answer.rename(columns= new_column)
answer['관측시점'] = answer['연도'].apply(convert_year) + '-' + answer['일시']
answer

# Merge the result DataFrame with the answer sheet
final_data = answer.merge(result[['관측시점', '지점', 'PM2.5']], on=['관측시점', '지점'], how='left')
final_data.drop(['PM2.5_x', '관측시점'], axis=1, inplace=True)
final_data.rename(columns={'PM2.5_y': 'PM2.5'}, inplace=True)
final_data.rename(columns={'지점': '측정소'}, inplace=True)
final_data

final_data.to_csv('/content/new/again.csv', index=False)

   연도           일시   지점  PM2.5              관측시점         위도          경도  \
0   0  01-01 00:00  성성동  0.044  2000-01-01 00:00  36.840313  127.141777   
1   0  01-01 00:00  신방동  0.056  2000-01-01 00:00  36.782355  127.120506   
2   0  01-01 01:00  성성동  0.040  2000-01-01 01:00  36.840313  127.141777   
3   0  01-01 01:00  신방동  0.048  2000-01-01 01:00  36.782355  127.120506   
4   0  01-01 02:00  성성동  0.048  2000-01-01 02:00  36.840313  127.141777   

  nearest_aws_지점    기온(°C)  풍향(deg)   풍속(m/s)  강수량(mm)  습도(%)  \
0             성거  0.157978   0.0125  0.012788      0.0  0.615   
1             성거  0.157978   0.0125  0.012788      0.0  0.615   
2             성거  0.153239   0.0000  0.007673      0.0  0.593   
3             성거  0.153239   0.0000  0.007673      0.0  0.593   
4             성거  0.137441   0.0000  0.010230      0.0  0.674   

           관측시점_aws  
0  2000-01-01 00:00  
1  2000-01-01 00:00  
2  2000-01-01 01:00  
3  2000-01-01 01:00  
4  2000-01-01 02:00  
   지점_pm      위도_pm       

<ipython-input-4-227e6112a2d4>:324: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_test_data_real = all_test_data_real.append(test_data_loc_real_df, ignore_index=True)


206/206 [==============================] - 1s 3ms/step
Location: 노은동, Train MAE: 0.044847634316269605, Validation MAE: 0.0370035892359741
2/2 [==============================] - 0s 7ms/step


<ipython-input-4-227e6112a2d4>:324: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_test_data_real = all_test_data_real.append(test_data_loc_real_df, ignore_index=True)


208/208 [==============================] - 1s 3ms/step
Location: 논산, Train MAE: 0.04885205323552174, Validation MAE: 0.04754688417266836
2/2 [==============================] - 0s 6ms/step


...

<ipython-input-4-227e6112a2d4>:324: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_test_data_real = all_test_data_real.append(test_data_loc_real_df, ignore_index=True)


212/212 [==============================] - 1s 3ms/step
Location: 정림동, Train MAE: 0.03822587576787157, Validation MAE: 0.03670023140409362
2/2 [==============================] - 0s 6ms/step


<ipython-input-4-227e6112a2d4>:324: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_test_data_real = all_test_data_real.append(test_data_loc_real_df, ignore_index=True)


210/210 [==============================] - 1s 3ms/step
Location: 홍성읍, Train MAE: 0.05638572589351701, Validation MAE: 0.04557658969243472
2/2 [==============================] - 0s 6ms/step


<ipython-input-4-227e6112a2d4>:324: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_test_data_real = all_test_data_real.append(test_data_loc_real_df, ignore_index=True)


In [ ]:
import pickle
with open('model.pkl', 'wb') as f:
    pickle.dump(model, f)

from google.colab import files

# Download the pickle file
files.download('model.pkl')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
files.download('/content/new/again.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
sorted_data

,지점,PM2.5,관측시점,기온(°C),풍향(deg),풍속(m/s),강수량(mm),습도(%)
270074,공주,0.056,2000-01-01 00:00,0.173776,0.201944,0.023018,0.0,0.828
270075,공주,0.060,2000-01-01 01:00,0.176935,0.168611,0.030691,0.0,0.831
270076,공주,0.068,2000-01-01 02:00,0.180095,0.087222,0.033248,0.0,0.784
270077,공주,0.060,2000-01-01 03:00,0.178515,0.087222,0.025575,0.0,0.745
270078,공주,0.068,2000-01-01 04:00,0.164297,0.113889,0.020460,0.0,0.750
...,...,...,...,...,...,...,...,...
568387,홍성읍,0.060,2003-12-31 19:00,0.273302,0.832222,0.086957,0.0,0.671
568388,홍성읍,0.052,2003-12-31 20:00,0.271722,0.831667,0.043478,0.0,0.692
568389,홍성읍,0.044,2003-12-31 21:00,0.268562,0.832500,0.066496,0.0,0.706
568390,홍성읍,0.052,2003-12-31 22:00,0.262243,0.866944,0.043478,0.0,0.725
